<a href="https://colab.research.google.com/github/BrandonKTC/Statistic_python/blob/main/RFM_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/MyDrive/Business Analyst course/Segmentation/RFM

/content/drive/MyDrive/Business Analyst course/Segmentation/RFM


In [2]:
# import libraries
import pandas as pd
from datetime import timedelta

In [23]:
data = pd.read_csv("customer_data.csv")
data.head()

,customer_id,revenue,most_recent_visit,number_of_orders,recency_days
0,22086,777,5/14/2006,9,232
1,2290,1555,9/8/2006,16,115
2,26377,336,11/19/2006,5,43
3,24650,1189,10/29/2006,12,64
4,12883,1229,12/9/2006,12,23


In [24]:
data["most_recent_visit"] = pd.to_datetime(data.most_recent_visit)

In [34]:
snapshot_date =  data.most_recent_visit.max() + timedelta(1)
df = data.drop('most_recent_visit', axis=1)
df['sales'] = df.revenue / df.number_of_orders
df.drop('revenue',axis=1, inplace=True)

In [36]:
# rename de column
df.rename({'number_of_orders': 'Frequency',
           'recency_days': 'Recency',
           'sales': 'Monetary'}, axis=1, inplace=True)

df.head()

,customer_id,Frequency,Recency,Monetary
0,22086,9,232,86.333333
1,2290,16,115,97.187500
2,26377,5,43,67.200000
3,24650,12,64,99.083333
4,12883,12,23,102.416667


In [46]:
df["F"] = pd.qcut(df.Frequency,
                  3,
                  range(1,4))
df["R"] = pd.qcut(df.Recency,
                  3,
                  range(1,4))
df["M"] = pd.qcut(df.Monetary,
                  3,
                  range(3,0,-1))

df["RFM_Score"] = df[["R","F","M"]].sum(axis=1)
df.head()

,customer_id,Frequency,Recency,Monetary,F,R,M,RFM_Score,RFM_level
0,22086,9,232,86.333333,2,3,3,8,Future Champion
1,2290,16,115,97.187500,3,2,2,7,Future Champion
2,26377,5,43,67.200000,1,1,3,5,High Potential
3,24650,12,64,99.083333,3,1,2,6,High Potential
4,12883,12,23,102.416667,3,1,2,6,High Potential


In [47]:
# Create the RFM Function
def rfm_segment(df):
  if df["RFM_Score"] >= 8:
    return "SuperStar"
  elif df["RFM_Score"] >= 5 and df["RFM_Score"] < 8:
    return "High Potential"
  else:
    return 'Low Relevance'

df["RFM_level"] = df.apply(rfm_segment, axis=1)

df.head()

,customer_id,Frequency,Recency,Monetary,F,R,M,RFM_Score,RFM_level
0,22086,9,232,86.333333,2,3,3,8,SuperStar
1,2290,16,115,97.187500,3,2,2,7,High Potential
2,26377,5,43,67.200000,1,1,3,5,High Potential
3,24650,12,64,99.083333,3,1,2,6,High Potential
4,12883,12,23,102.416667,3,1,2,6,High Potential


In [48]:
df.RFM_level.unique()

array(['SuperStar', 'High Potential', 'Low Relevance'], dtype=object)

In [49]:
# Aggregating the segments
df.groupby("RFM_level").agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', "count"]
})

Recency  Frequency    Monetary       
                      mean       mean        mean  count
RFM_level                                               
High Potential  187.171922   9.834542   94.424011  30479
Low Relevance    88.251001   7.844364  113.630475   5243
SuperStar       254.567688  12.239420   80.639399   4277